In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [31]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'c'
project = 'Joining Data in SQL'
data = make_dir_line(modality, project)

raw = data('raw')

In [32]:
import sqlite3

conn = sqlite3.connect(":memory:")  ## aca se indica el nombre de la db.
cur = conn.cursor()

# 6.1.0 Your first database

## 6.1.2 Attributes of relational databases

Which of the following statements does **not** hold true for databases? Relational databases

R:/ are called "relational" because they store data only about people.

## 6.1.3 Query information_schema with SELECT

In [33]:
conn.executescript(
    """
    
DROP TABLE IF EXISTS cities;

CREATE TABLE cities (
    name                  STRING,
    country_code          STRING,
    city_proper_pop       DOUBLE,
    metroarea_pop         DOUBLE,
    urbanarea_pop         DOUBLE
);


DROP TABLE IF EXISTS countries;

CREATE TABLE countries (
  code                  STRING     PRIMARY KEY,
  name                  VARCHAR,
  continent             VARCHAR,
  region                VARCHAR,
  surface_area          REAL,
  indep_year            INTEGER,
  local_name            VARCHAR,
  gov_form              VARCHAR,
  capital               VARCHAR,
  cap_long              REAL,
  cap_lat               REAL
);


DROP TABLE IF EXISTS economies;

CREATE TABLE economies (
  econ_id               INTEGER     PRIMARY KEY,
  code                  STRING,
  year                  INTEGER,
  income_group          VARCHAR,
  gdp_percapita         REAL,
  gross_savings         REAL,
  inflation_rate        REAL,
  total_investment      REAL,
  unemployment_rate     REAL,
  exports               REAL,
  imports               REAL
);


DROP TABLE IF EXISTS languages;

CREATE TABLE languages (
  lang_id               INTEGER     PRIMARY KEY,
  code                  VARCHAR,
  name                  VARCHAR,
  percent               REAL,
  official              BOOLEAN
);


DROP TABLE IF EXISTS populations;

CREATE TABLE populations (
  pop_id                DOUBLE     PRIMARY KEY,
  country_code          VARCHAR,
  year                  DOUBLE,
  fertility_rate        REAL,
  life_expectancy       REAL,
  size                  REAL
);



"""
)
conn.commit()

In [34]:
df = pd.read_csv(raw / 'countries/cities.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
cities = list(zip(df.name, df.country_code, df.city_proper_pop, df.metroarea_pop, df.urbanarea_pop))
cur.executemany("INSERT INTO cities VALUES  (?,?,?,?,?)", cities)

In [35]:
df = pd.read_csv(raw / 'countries/countries.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
countries = list(zip(df.code, df.country_name, df.continent, df.region, df.surface_area, df.indep_year, df.local_name, df.gov_form, df.capital, df.cap_long, df.cap_lat))
cur.executemany("INSERT INTO countries VALUES  (?,?,?,?,?,?,?,?,?,?,?)", countries)

In [36]:
df = pd.read_csv(raw / 'countries/economies.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
economies = list(zip(df.econ_id, df.code, df.year, df.income_group, df.gdp_percapita, df.gross_savings, df.inflation_rate, df.total_investment, df.unemployment_rate, df.exports, df.imports))
cur.executemany("INSERT INTO economies VALUES  (?,?,?,?,?,?,?,?,?,?,?)", economies)

In [37]:
df = pd.read_csv(raw / 'countries/languages.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
languages = list(zip(df.lang_id, df.code, df.name, df.percent, df.official))
cur.executemany("INSERT INTO languages VALUES  (?,?,?,?,?)", languages)

In [38]:
df = pd.read_csv(raw / 'countries/populations.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
populations = list(zip(df.pop_id, df.country_code, df.year, df.fertility_rate, df.life_expectancy, str(df.size)))
cur.executemany("INSERT INTO populations VALUES  (?,?,?,?,?,?)", populations)

## 5.1.2 Your first join

In [39]:
# -- Select all columns from cities
cur.execute("SELECT * FROM cities INNER JOIN countries ON cities.country_code = countries.code;").fetchall()

[('Abidjan',
  'CIV',
  4765000.0,
  None,
  4765000.0,
  'CIV',
  "Cote d'Ivoire",
  'Africa',
  'Western Africa',
  322463.0,
  1960,
  'Cote d\x92Ivoire',
  'Republic',
  'Yamoussoukro',
  -4.0305,
  5.332),
 ('Abu Dhabi',
  'ARE',
  1145000.0,
  None,
  1145000.0,
  'ARE',
  'United Arab Emirates',
  'Asia',
  'Middle East',
  83600.0,
  1971,
  'Al-Imarat al-´Arabiya al-Muttahida',
  'Emirate Federation',
  'Abu Dhabi',
  54.3705,
  24.4764),
 ('Abuja',
  'NGA',
  1235880.0,
  6000000.0,
  1235880.0,
  'NGA',
  'Nigeria',
  'Africa',
  'Western Africa',
  923768.0,
  1960,
  'Nigeria',
  'Federal Republic',
  'Abuja',
  7.48906,
  9.05804),
 ('Accra',
  'GHA',
  2070463.0,
  4010054.0,
  2070463.0,
  'GHA',
  'Ghana',
  'Africa',
  'Western Africa',
  238533.0,
  1957,
  'Ghana',
  'Republic',
  'Accra',
  -0.20795,
  5.57045),
 ('Addis Ababa',
  'ETH',
  3103673.0,
  4567857.0,
  3103673.0,
  'ETH',
  'Ethiopia',
  'Africa',
  'Eastern Africa',
  1104300.0,
  -1000,
  'YeItyop´iy

In [40]:
# -- Select name fields (with alias) and region 
cur.execute("SELECT cities.name AS city, countries.name AS country, region FROM cities INNER JOIN countries ON cities.country_code = countries.code;").fetchall()

[('Abidjan', "Cote d'Ivoire", 'Western Africa'),
 ('Abu Dhabi', 'United Arab Emirates', 'Middle East'),
 ('Abuja', 'Nigeria', 'Western Africa'),
 ('Accra', 'Ghana', 'Western Africa'),
 ('Addis Ababa', 'Ethiopia', 'Eastern Africa'),
 ('Ahmedabad', 'India', 'Southern and Central Asia'),
 ('Alexandria', 'Egypt', 'Northern Africa'),
 ('Algiers', 'Algeria', 'Northern Africa'),
 ('Almaty', 'Kazakhstan', 'Southern and Central Asia'),
 ('Ankara', 'Turkey', 'Middle East'),
 ('Auckland', 'New Zealand', 'Australia and New Zealand'),
 ('Baghdad', 'Iraq', 'Middle East'),
 ('Baku', 'Azerbaijan', 'Middle East'),
 ('Bandung', 'Indonesia', 'Southeast Asia'),
 ('Bangkok', 'Thailand', 'Southeast Asia'),
 ('Barcelona', 'Spain', 'Southern Europe'),
 ('Barranquilla', 'Colombia', 'South America'),
 ('Basra', 'Iraq', 'Middle East'),
 ('Beijing', 'China', 'Eastern Asia'),
 ('Belo Horizonte', 'Brazil', 'South America'),
 ('Bengaluru', 'India', 'Southern and Central Asia'),
 ('Berlin', 'Germany', 'Western Europe

## 5.1.3 Joining with aliased tables

In [41]:
# -- Select fields with aliases, Join to economies (alias e), Match on code field using table aliases
cur.execute("SELECT c.code AS country_code, name, year, inflation_rate FROM countries AS c INNER JOIN economies AS e ON e.code = c.code;").fetchall()

[('AFG', 'Afghanistan', 2010, 2.179),
 ('AFG', 'Afghanistan', 2015, -1.549),
 ('AGO', 'Angola', 2010, 14.48),
 ('AGO', 'Angola', 2015, 10.287),
 ('ALB', 'Albania', 2010, 3.605),
 ('ALB', 'Albania', 2015, 1.896),
 ('ARE', 'United Arab Emirates', 2010, 0.878),
 ('ARE', 'United Arab Emirates', 2015, 4.07),
 ('ARG', 'Argentina', 2010, 10.461),
 ('ARG', 'Argentina', 2015, None),
 ('ARM', 'Armenia', 2010, 7.274),
 ('ARM', 'Armenia', 2015, 3.731),
 ('ATG', 'Antigua and Barbuda', 2010, 3.37),
 ('ATG', 'Antigua and Barbuda', 2015, 0.969),
 ('AUS', 'Australia', 2010, 2.863),
 ('AUS', 'Australia', 2015, 1.461),
 ('AUT', 'Austria', 2010, 1.694),
 ('AUT', 'Austria', 2015, 0.81),
 ('AZE', 'Azerbaijan', 2010, 5.666),
 ('AZE', 'Azerbaijan', 2015, 4.049),
 ('BDI', 'Burundi', 2010, 6.496),
 ('BDI', 'Burundi', 2015, 5.553),
 ('BEL', 'Belgium', 2010, 2.334),
 ('BEL', 'Belgium', 2015, 0.62),
 ('BEN', 'Benin', 2010, 2.179),
 ('BEN', 'Benin', 2015, 0.271),
 ('BFA', 'Burkina Faso', 2010, -0.608),
 ('BFA', 'Bu

## 5.1.4 USING in action

In [42]:
# -- Match using the code column
cur.execute("SELECT c.name AS country, l.name AS language, official FROM countries AS c INNER JOIN languages AS l USING(code);").fetchall()

[('Afghanistan', 'Dari', 1),
 ('Afghanistan', 'Pashto', 1),
 ('Afghanistan', 'Turkic', 0),
 ('Afghanistan', 'Other', 0),
 ('Albania', 'Albanian', 1),
 ('Albania', 'Greek', 0),
 ('Albania', 'Other', 0),
 ('Albania', 'unspecified', 0),
 ('Algeria', 'Arabic', 1),
 ('Algeria', 'French', 0),
 ('Algeria', 'Berber or Tamazight', 1),
 ('Algeria', 'Shawiya', 0),
 ('Algeria', 'Mzab', 0),
 ('Algeria', 'Tuareg', 0),
 ('American Samoa', 'Samoan', 0),
 ('American Samoa', 'English', 0),
 ('American Samoa', 'Tongan', 0),
 ('American Samoa', 'Other', 0),
 ('American Samoa', 'Other', 0),
 ('Andorra', 'Catalan', 1),
 ('Andorra', 'French', 0),
 ('Andorra', 'Castilian', 0),
 ('Andorra', 'Portuguese', 0),
 ('Angola', 'Portuguese', 1),
 ('Angola', 'Umbundu', 0),
 ('Angola', 'Kikongo', 0),
 ('Angola', 'Kimbundu', 0),
 ('Angola', 'Chokwe', 0),
 ('Angola', 'Nhaneca', 0),
 ('Angola', 'Nganguela', 0),
 ('Angola', 'Fiote', 0),
 ('Angola', 'Kwanhama', 0),
 ('Angola', 'Muhumbi', 0),
 ('Angola', 'Luvale', 0),
 ('Ango

## 5.1.6 Relationships in our database

In [43]:
df = pd.read_csv(raw / 'countries/countries.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
print('Unicidad', df['code'].nunique() == df.shape[0])
print('longitud data', df.shape)
print('Codigos unicos', df['code'].nunique())

Unicidad True
longitud data (205, 11)
Codigos unicos 205


In [44]:
df = pd.read_csv(raw / 'countries/cities.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
print('Unicidad', df['country_code'].nunique() == df.shape[0])
print('longitud data', df.shape)
print('Codigos unicos', df['country_code'].nunique())

Unicidad False
longitud data (236, 5)
Codigos unicos 81


Question
What best describes the relationship between code in the countries table and country_code in the cities table?

R:/ This is a one to many relationship

In [45]:
df = pd.read_csv(raw / 'countries/countries.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
print('Unicidad', df['code'].nunique() == df.shape[0])
print('longitud data', df.shape)
print('Codigos unicos', df['code'].nunique())

Unicidad True
longitud data (205, 11)
Codigos unicos 205


In [46]:
df = pd.read_csv(raw / 'countries/languages.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
print('Unicidad', df['code'].nunique() == df.shape[0])
print('longitud data', df.shape)
print('Codigos unicos', df['code'].nunique())

Unicidad False
longitud data (955, 5)
Codigos unicos 212


Question
What best describes the relationship between code in the countries table and country_code in the cities table?

R:/ This is a many to many relationship  (is one to many but one country can have one or more languages)

## 5.1.7 Inspecting a relationship

In [47]:
# -- Select country and language names, aliased, From countries (aliased), Join to languages (aliased), Use code as the joining field with the USING keyword
cur.execute("SELECT c.name AS country, l.name AS language FROM countries AS c INNER JOIN languages AS l USING (code);").fetchall()

[('Afghanistan', 'Dari'),
 ('Afghanistan', 'Pashto'),
 ('Afghanistan', 'Turkic'),
 ('Afghanistan', 'Other'),
 ('Albania', 'Albanian'),
 ('Albania', 'Greek'),
 ('Albania', 'Other'),
 ('Albania', 'unspecified'),
 ('Algeria', 'Arabic'),
 ('Algeria', 'French'),
 ('Algeria', 'Berber or Tamazight'),
 ('Algeria', 'Shawiya'),
 ('Algeria', 'Mzab'),
 ('Algeria', 'Tuareg'),
 ('American Samoa', 'Samoan'),
 ('American Samoa', 'English'),
 ('American Samoa', 'Tongan'),
 ('American Samoa', 'Other'),
 ('American Samoa', 'Other'),
 ('Andorra', 'Catalan'),
 ('Andorra', 'French'),
 ('Andorra', 'Castilian'),
 ('Andorra', 'Portuguese'),
 ('Angola', 'Portuguese'),
 ('Angola', 'Umbundu'),
 ('Angola', 'Kikongo'),
 ('Angola', 'Kimbundu'),
 ('Angola', 'Chokwe'),
 ('Angola', 'Nhaneca'),
 ('Angola', 'Nganguela'),
 ('Angola', 'Fiote'),
 ('Angola', 'Kwanhama'),
 ('Angola', 'Muhumbi'),
 ('Angola', 'Luvale'),
 ('Angola', 'Other'),
 ('Antigua and Barbuda', 'English'),
 ('Antigua and Barbuda', 'Antiguan creole'),
 ('Ar

In [48]:
# -- Rearrange SELECT statement, keeping aliases, Order the results by language
cur.execute("SELECT c.name AS country, l.name AS language FROM countries AS c INNER JOIN languages AS l USING(code) ORDER BY language;").fetchall()

[('Djibouti', 'Afar'),
 ('Eritrea', 'Afar'),
 ('Ethiopia', 'Afar'),
 ('Namibia', 'Afrikaans'),
 ('South Africa', 'Afrikaans'),
 ('Ghana', 'Akyem'),
 ('Albania', 'Albanian'),
 ('Macedonia', 'Albanian'),
 ('Switzerland', 'Albanian'),
 ('France', 'Alsatian'),
 ('Guatemala', 'Amerindian'),
 ('Guyana', 'Amerindian'),
 ('Honduras', 'Amerindian'),
 ('Ethiopia', 'Amharic'),
 ('Sao Tome and Principe', 'Angolar'),
 ('Antigua and Barbuda', 'Antiguan creole'),
 ('Algeria', 'Arabic'),
 ('Australia', 'Arabic'),
 ('Bahrain', 'Arabic'),
 ('Canada', 'Arabic'),
 ('Chad', 'Arabic'),
 ('Comoros', 'Arabic'),
 ('Cyprus', 'Arabic'),
 ('Djibouti', 'Arabic'),
 ('Egypt', 'Arabic'),
 ('Eritrea', 'Arabic'),
 ('Ethiopia', 'Arabic'),
 ('Iran', 'Arabic'),
 ('Iraq', 'Arabic'),
 ('Israel', 'Arabic'),
 ('Jordan', 'Arabic'),
 ('Kuwait', 'Arabic'),
 ('Lebanon', 'Arabic'),
 ('Libya', 'Arabic'),
 ('Mauritania', 'Arabic'),
 ('Morocco', 'Arabic'),
 ('Oman', 'Arabic'),
 ('Panama', 'Arabic'),
 ('Qatar', 'Arabic'),
 ('Saudi Ara

Question
Select the incorrect answer from the following options.

The query you generated in step 1 is provided below. Run this query (or the amendment you made in step 2) in the console to find the answer to the question.

R:/ Alsatian is only spoken in France.

## 5.1.9 Joining multiple tables

In [49]:
# -- Select relevant fields, Inner join countries and y, aliased, on code, Join to economies (as e)
cur.execute("SELECT name, e.year, fertility_rate, e.unemployment_rate FROM countries AS c INNER JOIN populations AS p ON c.code = p.country_code INNER JOIN economies AS e ON c.code = e.code;").fetchall()

[('Afghanistan', 2010, 5.746, None),
 ('Afghanistan', 2015, 5.746, None),
 ('Afghanistan', 2010, 4.653, None),
 ('Afghanistan', 2015, 4.653, None)]

## 5.1.10 Checking multi-table joins

In [50]:
# -- Add an additional joining condition such that you are also joining on year
cur.execute("SELECT name, e.year, fertility_rate, unemployment_rate FROM countries AS c INNER JOIN populations AS p ON c.code = p.country_code INNER JOIN economies AS e ON c.code = e.code AND e.year = p.year;").fetchall()

[('Afghanistan', 2010, 5.746, None), ('Afghanistan', 2015, 4.653, None)]

In [51]:
print('Ok_')

Ok_
